<a href="https://colab.research.google.com/github/lchutny/MortgageCalcs/blob/main/MortgagePayoutCalc_July2021.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

In [2]:
!pip install numpy_financial

In [125]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import math
import sys
import os
from os import listdir, fspath
from os.path import isfile, join
import numpy_financial as npf
from datetime import date
from collections import OrderedDict
from dateutil.relativedelta import *

print (f"Python version: {sys.version}")
print(f"Pandas Version: {pd.__version__}")
print(f"Numpy version: {np.__version__}")
print(f"Numpy Financial Version {npf.__version__}")

Python version: 3.7.11 (default, Jul  3 2021, 18:01:19) 
[GCC 7.5.0]
Pandas Version: 1.1.5
Numpy version: 1.19.5
Numpy Financial Version 1.0.0


# Input Data

In [18]:
def calc_r(postedrate, periodsperyear):
    """Calculates the effective annual rate for a Canadian Posted Rate based on Canada's legal Semi Annual Compounding
    and an input number of payment periods per year
    """
    semiannualrate = postedrate/2
    EAR =((1+semiannualrate)**2-1)  # Effective Annual Rate, EAR
    reff_perperiod=(1+EAR)**(1/periodsperyear)-1  # EAR per period
    apr = reff_perperiod*periodsperyear # back calculate the annual percentage rate - a yearly rate based on the semi annual compounding
    return apr,EAR

In [28]:
P = 100000 # principal
postedrate = 0.05 # annual interest rate - as quoted
n = 12 # number of payments per year
apr,EAR = calc_r(postedrate,n)
t = 25 # amortization period, years
startdate = (date(2021,1,1))
Addl_Principal = 0 # Additional Principal per period

#### Change Rate for Canada - Semi Annual Compounding
From https://vindeep.com/Corporate/InterestRateConversion.aspx and http://www.yorku.ca/amarshal/mortgage.htm   
(In second reference, note error in 2nd line of page, where it should be $r_M = (1.0609)^{1/12}-1$    

Note dox for numpy financial:  
https://numpy.org/numpy-financial/latest/index.html

# Basic Calculations

In [29]:
r_n = apr/n  # interest rate per period
totpymts = n*t # total number of payments
pymt = npf.pmt(r_n,totpymts,P)  # Calculate Payment per period
per = 1 # Payment period to calculate the interest amount
ipmt = npf.ipmt(r_n,per,totpymts,P)
ppmt = npf.ppmt(r_n,per,totpymts,P)
print(f"Principal of ${P}, {n} payments/year, {postedrate*100}% interest as posted, {t} years amortization, payments are ${-pymt:.2f} per period.")
print(f"APR in Canada: for a posted rate of {postedrate*100}%, APR = {apr*100:.4f}% and the EAR ={EAR*100:.4f}%")
print(f"For period {per}, the interest payment is ${-ipmt:.2f} and the principal payment is ${-ppmt:.2f}, not counting for additional payments per period of ${Addl_Principal:.2f}")

Principal of $100000, 12 payments/year, 5.0% interest as posted, 25 years amortization, payments are $581.60 per period.
APR in Canada: for a posted rate of 5.0%, APR = 4.9487% and the EAR =5.0625%
For period 1, the interest payment is $412.39 and the principal payment is $169.21, not counting for additional payments per period of $0.00


# Build Amortization Table
From https://pbpython.com/amortization-model-revised.html

### Create generator

In [87]:
def amortize(principal, interest_rate, years, pmt, addl_principal=0, annual_payments=12, start_date=date.today()):
    """Generator to calculate the Amortization table
    This can be used to generate the dataframe
    
    Inputs are:
    principal = Principal amount
    interest_rate = posted annual interest rate
    years = years of amortization, 
    pmt = payment amount per period
    addl_principal = Additional Principal paid per period
    annual_payments = annual number of regular payments
    start_date = 
    start date
    
    Output is Ordered Dictionary of results
    """
        
    # initialize the variables to keep track of the periods and running balances
    p = 1
    beg_balance = principal
    end_balance = principal

    while end_balance > 0:

        # Recalculate the interest based on the current balance
        interest = round(((interest_rate/annual_payments) * beg_balance), 2)

        # Determine payment based on whether or not this period will pay off the loan
        pmt = min(pmt, beg_balance + interest)
        principal = pmt - interest

        # Ensure additional payment gets adjusted if the loan is being paid off
        addl_principal = min(addl_principal, beg_balance - principal)
        end_balance = beg_balance - (principal + addl_principal)

        yield OrderedDict([('Month',start_date),
                           ('Period', p),
                           ('BeginBal', beg_balance),
                           ('Payment', pmt),
                           ('Principal', principal),
                           ('Interest', interest),
                           ('AddtnlPayment', addl_principal),
                           ('EndBal', end_balance)])

        # Increment the counter, balance and date
        p += 1
        start_date += relativedelta(months=1)
        beg_balance = end_balance
        if p > 1000:
          break


### Create Amortization Table

In [77]:
def AmortTable(principal, interest_rate, years, pmt=0,
                       addl_principal=0, annual_payments=12, start_date=date.today()):
    """
    Calculate the amortization schedule given the loan details as well as summary stats for the loan

    :param principal: Amount borrowed
    :param interest_rate: The annual interest rate for this loan
    :param years: Number of years for the loan
    :param pmt: the fixed payment, if zero, calculate payment in this function
    
    :param annual_payments (optional): Number of payments in a year. DEfault 12.
    :param addl_principal (optional): Additional payments to be made each period. Default 0.
    :param start_date (optional): Start date. Default first of next month if none provided

    :return: 
        schedule: Amortization schedule as a pandas dataframe
        summary: Pandas dataframe that summarizes the payoff information
    """
    
    # Payment stays constant based on the original terms of the loan
    if pmt == 0:
      payment = -round(npf.pmt(interest_rate/annual_payments, years*annual_payments, principal), 2)
    else:
      payment=-pmt
    
    # Generate the schedule and order the resulting columns for convenience
    schedule = pd.DataFrame(amortize(principal, interest_rate, years, payment,
                                     addl_principal, annual_payments, start_date))
    schedule = schedule[["Period", "Month", "BeginBal", "Payment", "Principal", 
                          "Interest", "AddtnlPayment", "EndBal"]]
    
    # Convert to a datetime object to make subsequent calcs easier
    schedule["Month"] = pd.to_datetime(schedule["Month"])
    
    #Create a summary statistics table
    payoff_date = schedule["Month"].iloc[-1]
    posted_rt = 2*((1+interest_rate/12)**6-1)
    stats = pd.Series([payoff_date, schedule["Period"].count(), round(interest_rate*100,2),
                       round(posted_rt*100,2),years, principal, payment, addl_principal,
                       schedule["Interest"].sum()],
                       index=["Payoff Date", "Num Payments", "Effective Annual Interest Rate, %", 
                              "Posted Annual Rate, %","Amortization Period, Years", "Principal",
                             "Payment", "Additional Payment", "Total Interest"])
    
    return schedule, stats

### Get Results

In [78]:
sch,stats = AmortTable(principal=P,interest_rate=apr,years=t,pmt=0,addl_principal=Addl_Principal,annual_payments=n,start_date=startdate)

sch.head()

,Period,Month,BeginBal,Payment,Principal,Interest,AddtnlPayment,EndBal
0,1,2021-01-01,100000.00,581.6,169.21,412.39,0,99830.79
1,2,2021-02-01,99830.79,581.6,169.91,411.69,0,99660.88
2,3,2021-03-01,99660.88,581.6,170.61,410.99,0,99490.27
3,4,2021-04-01,99490.27,581.6,171.31,410.29,0,99318.96
4,5,2021-05-01,99318.96,581.6,172.02,409.58,0,99146.94


In [79]:
sch.tail()

,Period,Month,BeginBal,Payment,Principal,Interest,AddtnlPayment,EndBal
296,297,2045-09-01,2305.52,581.60,572.09,9.51,0,1733.43
297,298,2045-10-01,1733.43,581.60,574.45,7.15,0,1158.98
298,299,2045-11-01,1158.98,581.60,576.82,4.78,0,582.16
299,300,2045-12-01,582.16,581.60,579.20,2.40,0,2.96
300,301,2046-01-01,2.96,2.97,2.96,0.01,0,0.00


The above schedule for 100,000 at 5% amortized over 25 years with $581.60 payments monthly matches the online calculator exactly (see the pdf). So the amortization schedule is being done correctly.

In [80]:
stats

Payoff Date                          2046-01-01 00:00:00
Num Payments                                         301
Effective Annual Interest Rate, %                   4.95
Posted Annual Rate, %                                  5
Amortization Period, Years                            25
Principal                                         100000
Payment                                            581.6
Additional Payment                                     0
Total Interest                                     74483
dtype: object

# Scenarios

### 1. First Year of Repayment
2021 - no extra payments. Determine principal owing at end of Dec 2021

In [174]:
P2 = 449878.87 # principal at start of 2021
postedrate2 = 0.019584 # annual interest rate - as quoted - annual interest rate on Mortgage Docs
n = 12 # number of payments per year
apr2,ear = calc_r(postedrate2,n)
t2 = 23.5 # amortization period, years
startdate = (date(2021,1,1))
Addl_Principal = 0 # Additional Principal per period fpr 2021
r_n2 = apr2/n  # monthly interest rate
totpymts2 = n*t2 # total number of payments

In [175]:
pymt2 = npf.pmt(r_n2,totpymts2,P2)  # Calculate Monthly Payment
per = 1 # Payment period to calculate the interest amount
ipmt2 = npf.ipmt(r_n2,per,totpymts2,P2)
ppmt2 = npf.ppmt(r_n2,per,totpymts2,P2)
print(f"Principal of ${P2}, {n} payments/year, {postedrate2*100:.4f}% interest as posted, {t2} years amortization, required payments are ${-pymt2:.2f} per period.")
print(f"APR in Canada: for a posted rate of {postedrate2*100:.4f}%, APR = {apr2*100:.4f}% and the EAR ={ear*100:.4f}%")
print(f"For period {per}, the required interest payment is ${-ipmt2:.2f} and the required principal payment is ${-ppmt2:.2f}, not counting for additional payments per period of ${Addl_Principal:.2f}")

Principal of $449878.87, 12 payments/year, 1.9584% interest as posted, 23.5 years amortization, required payments are $1990.03 per period.
APR in Canada: for a posted rate of 1.9584%, APR = 1.9505% and the EAR =1.9680%
For period 1, the required interest payment is $731.22 and the required principal payment is $1258.81, not counting for additional payments per period of $0.00


This calculation matches the mortgage docs for payment amount, amortization, posted rate and principal. So likely correct. However, the docs also show an 'Annual Percentage Rate' of 1.94362% - and I have no idea how they get this.

In [176]:
sch2,stats2 = AmortTable(principal=P2,interest_rate=apr2,years=t2,pmt=0,addl_principal=Addl_Principal,annual_payments=n,start_date=startdate)

In [177]:
sch2[11:16]

,Period,Month,BeginBal,Payment,Principal,Interest,AddtnlPayment,EndBal
11,12,2021-12-01,435918.92,1990.03,1281.50,708.53,0,434637.42
12,13,2022-01-01,434637.42,1990.03,1283.58,706.45,0,433353.84
13,14,2022-02-01,433353.84,1990.03,1285.66,704.37,0,432068.18
14,15,2022-03-01,432068.18,1990.03,1287.75,702.28,0,430780.43
15,16,2022-04-01,430780.43,1990.03,1289.85,700.18,0,429490.58


In [83]:
begApr2022_bal = sch2.iloc[15,2]
print(begApr2022_bal)

430780.62999999995


### 2. Second Year of Repayment  
Assumes:
\$250,000 extra available for the prepayments from settlement of 2836 address;  
2021 - no extra payments  
2022 - repayment of 15% of original principal in Month 3 - apply to Month 4  
2022 - double up all payments for Months 4-12  

In [71]:
Tot_AddlPrincipal = 250000  # From sale of house
Addl_Principal2_yearly = 0.15*P2 # additional amount once peryear
Addl_monthly_Payment = pymt2
print(Addl_Principal2_yearly,Addl_monthly_Payment)

67481.8305 -1990.0533397467575


In [89]:
P3 = begApr2022_bal - Addl_Principal2_yearly # principal at start of April 2022 less the lump sum for 2022
t3 = t2-15/12 # amortization period, years from April 2022 - end
startdate3 = (date(2022,4,1))
Addl_Principal3 = -Addl_monthly_Payment # Additional Principal per period for 2022 - Match a Payment Option
totpymts3 = n*t3 # total number of payments

In [75]:
print(t3,startdate3,Addl_Principal3,totpymts3)

22.25 2022-04-01 -1990.0533397467575 267.0


In [90]:
sch3,stats3 = AmortTable(principal=P3,interest_rate=apr2,years=t3,pmt=pymt2,addl_principal=Addl_Principal3,annual_payments=n,start_date=startdate3)

In [92]:
sch3[7:10]

,Period,Month,BeginBal,Payment,Principal,Interest,AddtnlPayment,EndBal
7,8,2022-11-01,339455.722744,1990.05334,1438.28334,551.77,1990.05334,336027.386064
8,9,2022-12-01,336027.386064,1990.05334,1443.85334,546.20,1990.05334,332593.479385
9,10,2023-01-01,332593.479385,1990.05334,1449.43334,540.62,1990.05334,329153.992705


In [93]:
begJan2023_bal = sch3.iloc[9,2]
print(begJan2023_bal)

332593.47938455833


### 3. Third Year of Repayment  
Assumes:
\$250,000 extra available for the prepayments from settlement of 2836 address;  
2023 - repayment of 15% of original principal in Month 1 - apply to Month 1  
2023 - double up all payments for all months

In [116]:
#Check how much additional principal left at start of year:
Tot_AddlPrincipal_left = 250000-Addl_Principal2_yearly-(-9*Addl_monthly_Payment)  # From sale of house
print("Additional Principal left at start of 2023 = ",Tot_AddlPrincipal_left)

Additional Principal left at start of 2023 =  164607.6894422792


In [99]:

Addl_Principal2_yearly = 0.15*P2 # additional amount once peryear
Addl_monthly_Payment = pymt2
print(Addl_Principal2_yearly,Addl_monthly_Payment)

67481.8305 -1990.0533397467575


In [100]:
P4 = begJan2023_bal - Addl_Principal2_yearly # principal at start of Jan 2023 less the lump sum for 2023
t4 = t3-9/12 # amortization period, years from Jan 2023 - end
startdate4 = (date(2023,1,1))
Addl_Principal3 = -Addl_monthly_Payment # Additional Principal per period for 2023 - Match a Payment Option
totpymts4 = n*t4 # total number of payments

In [101]:
print(t4,startdate4,Addl_Principal3,totpymts4)

21.5 2023-01-01 1990.0533397467575 258.0


In [102]:
sch4,stats4 = AmortTable(principal=P4,interest_rate=apr2,years=t4,pmt=pymt2,addl_principal=Addl_Principal3,annual_payments=n,start_date=startdate4)

In [105]:
sch4[9:13]

,Period,Month,BeginBal,Payment,Principal,Interest,AddtnlPayment,EndBal
9,10,2023-10-01,232960.558769,1990.05334,1611.38334,378.67,1990.05334,229359.122090
10,11,2023-11-01,229359.122090,1990.05334,1617.24334,372.81,1990.05334,225751.825410
11,12,2023-12-01,225751.825410,1990.05334,1623.10334,366.95,1990.05334,222138.668731
12,13,2024-01-01,222138.668731,1990.05334,1628.97334,361.08,1990.05334,218519.642051


In [106]:
begJan2024_bal = sch4.iloc[12,2]
print(begJan2024_bal)

222138.66873063613


### 4. Fourth Year of Repayment - first two months 
Assumes:
\$250,000 extra available for the prepayments from settlement of 2836 address;  
2024 - repayment of 15% of original principal in Month 1 - apply to Month 1  
2024 - double up all payments for months as long as principal lasts

In [117]:
#Check how much additional principal left:
Tot_AddlPrincipal_left4 = Tot_AddlPrincipal_left-Addl_Principal2_yearly-(-12*Addl_monthly_Payment)  # From sale of house
print("Total Additional Principal Left at start of 2024 = ",Tot_AddlPrincipal_left4)

Total Additional Principal Left at start of 2024 =  73245.21886531811


So we can't double up all payments, only to the limit of the principal. Use the leftover amount in year 5 as a lump sum payment

In [127]:
tap4_mthly= Tot_AddlPrincipal_left4 - Addl_Principal2_yearly
print("Available funds for monthly double ups = ",tap4_mthly)
mths = math.floor(tap4_mthly/-pymt2)
print("Months available to match payment =",mths)
leftover = tap4_mthly - mths*(-pymt2)
print("Leftover $ =",leftover)

Available funds for monthly double ups =  5763.388365318111
Months available to match payment = 2
Leftover $ = 1783.281685824596


In [108]:
Addl_Principal2_yearly = 0.15*P2 # additional amount once peryear
Addl_monthly_Payment = pymt2
print(Addl_Principal2_yearly,Addl_monthly_Payment)

67481.8305 -1990.0533397467575


In [130]:
P5 = begJan2024_bal - Addl_Principal2_yearly # principal at start of Jan 2024 less the lump sum for 2024
t5 = t4-1 # amortization period, years from Jan 2024 - end
startdate5 = (date(2024,1,1))
Addl_Principal3 = -Addl_monthly_Payment # Additional Principal per period for 2024 first two months - Match a Payment Option
totpymts5 = n*t5 # total number of payments

In [131]:
print(t5,startdate5,Addl_Principal3,totpymts5)

20.5 2024-01-01 1990.0533397467575 246.0


In [132]:
sch5,stats5 = AmortTable(principal=P5,interest_rate=apr2,years=t5,pmt=pymt2,addl_principal=Addl_Principal3,annual_payments=n,start_date=startdate5)

In [134]:
sch5[2:6]

,Period,Month,BeginBal,Payment,Principal,Interest,AddtnlPayment,EndBal
2,3,2024-03-01,147193.344872,1990.05334,1750.79334,239.26,1990.05334,143452.498192
3,4,2024-04-01,143452.498192,1990.05334,1756.87334,233.18,1990.05334,139705.571513
4,5,2024-05-01,139705.571513,1990.05334,1762.96334,227.09,1990.05334,135952.554833
5,6,2024-06-01,135952.554833,1990.05334,1769.07334,220.98,1990.05334,132193.428154


In [136]:
begMar2024_bal = sch5.iloc[2,2]
print(begMar2024_bal)

147193.3448716491


### 4A. Months 3-12 2024 - Fourth Year of Repayment  
Assumes:
\$250,000 extra available for the prepayments from settlement of 2836 address;  
2024 Months 3-12 - just standard amortization

In [138]:
P5A = begMar2024_bal # principal at start of March 2024
t5A = t5-2/12 # amortization period, years from March 2024 - end
startdate5A = (date(2024,3,1))
Addl_Principal5A = 0 # No additional Monthly principal left
totpymts5A = n*t5A # total number of payments

In [139]:
print(t5A,startdate5A,Addl_Principal5A,totpymts5A)

20.333333333333332 2024-03-01 0 244.0


In [141]:
sch5A,stats5A = AmortTable(principal=P5A,interest_rate=apr2,years=t5A,pmt=pymt2,addl_principal=Addl_Principal5A,annual_payments=n,start_date=startdate5A)

In [142]:
sch5A[9:13]

,Period,Month,BeginBal,Payment,Principal,Interest,AddtnlPayment,EndBal
9,10,2024-12-01,131333.334814,1990.05334,1776.57334,213.48,0,129556.761474
10,11,2025-01-01,129556.761474,1990.05334,1779.46334,210.59,0,127777.298134
11,12,2025-02-01,127777.298134,1990.05334,1782.35334,207.70,0,125994.944795
12,13,2025-03-01,125994.944795,1990.05334,1785.25334,204.80,0,124209.691455


In [143]:
begJan2025_bal = sch5A.iloc[10,2]
print(begJan2025_bal)

129556.7614741815


### 5. Fifth Year of Repayment - 2025 - Last Year

Assumes:
2025 - use leftover amount from 2024 monthlies in Month 1 - apply to Month 1  

In [144]:
Addl_Principal6_yearly = leftover # additional amount once peryear
Addl_monthly_Payment6 = 0
print(Addl_Principal6_yearly,Addl_monthly_Payment6)

1783.281685824596 0


In [145]:
P6 = begJan2025_bal - Addl_Principal6_yearly # principal at start of Jan 2025 less the lump sum for 2025
t6 = t5-1 # amortization period, years from Jan 2025 - end
startdate6 = (date(2025,1,1))
Addl_Principal6 = Addl_monthly_Payment6 # Additional Principal per period for 2025 - None
totpymts6 = n*t6 # total number of payments

In [146]:
print(t6,startdate6,Addl_Principal6,totpymts6)

19.5 2025-01-01 0 234.0


In [147]:
sch6,stats6 = AmortTable(principal=P6,interest_rate=apr2,years=t6,pmt=pymt2,addl_principal=Addl_Principal6,annual_payments=n,start_date=startdate6)

In [148]:
sch6[9:13]

,Period,Month,BeginBal,Payment,Principal,Interest,AddtnlPayment,EndBal
9,10,2025-10-01,111627.509731,1990.05334,1808.60334,181.45,0,109818.906391
10,11,2025-11-01,109818.906391,1990.05334,1811.54334,178.51,0,108007.363051
11,12,2025-12-01,108007.363051,1990.05334,1814.49334,175.56,0,106192.869711
12,13,2026-01-01,106192.869711,1990.05334,1817.44334,172.61,0,104375.426372


In [ ]:
begJan2025_bal = sch5.iloc[12,2]
print(begJan2025_bal)

109510.00807671394


### 5A. Fifth Year of Repayment - 2025 - Last Year - Alternate Scenario

Assumes:
2025 - use leftover amount from 2024 monthlies in Month 1 - apply to Month 1  
Determine how much extra lump sum we could/should use to get to < \$20,000 by end of year

In [157]:
EXTRA = 80000
Addl_Principal6A_yearly = leftover+EXTRA # additional amount once peryear
Addl_monthly_Payment6A = 0
print(Addl_Principal6A_yearly,Addl_monthly_Payment6A)

81783.2816858246 0


In [158]:
P6A = begJan2025_bal - Addl_Principal6A_yearly # principal at start of Jan 2025 less the lump sum for 2025
t6A = t5-1 # amortization period, years from Jan 2025 - end
startdate6A = (date(2025,1,1))
Addl_Principal6A = Addl_monthly_Payment6A # Additional Principal per period for 2025 - None
totpymts6A = n*t6A # total number of payments

In [159]:
print(t6A,startdate6A,Addl_Principal6A,totpymts6A)

19.5 2025-01-01 0 234.0


In [160]:
sch6A,stats6A = AmortTable(principal=P6A,interest_rate=apr2,years=t6A,pmt=pymt2,addl_principal=Addl_Principal6A,annual_payments=n,start_date=startdate6A)

In [161]:
sch6A[9:13]

,Period,Month,BeginBal,Payment,Principal,Interest,AddtnlPayment,EndBal
9,10,2025-10-01,30449.549731,1990.05334,1940.56334,49.49,0,28508.986391
10,11,2025-11-01,28508.986391,1990.05334,1943.71334,46.34,0,26565.273051
11,12,2025-12-01,26565.273051,1990.05334,1946.87334,43.18,0,24618.399711
12,13,2026-01-01,24618.399711,1990.05334,1950.03334,40.02,0,22668.366372
